In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch
import torchaudio
import os
import matplotlib.pyplot as plt
import librosa
from IPython.display import Audio
import librosa
import matplotlib.pyplot as plt
from torchaudio.utils import download_asset
import cv2
import glob
import pandas as pd


from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn.functional as F
from torch import nn
import torch
import torch.optim as optim
import lightning.pytorch as pl
from torchaudio.transforms import *
import numpy as np
import torchmetrics
from lightning.pytorch.loggers import CSVLogger
from torch.nn.utils.rnn import pad_sequence
import re
from transformers import AutoFeatureExtractor, ASTForAudioClassification

from torch.utils.data import DataLoader
from emotion_utils.utils.utils import *

In [3]:
llogger = CSVLogger("emotions", 'classification-lm')

In [4]:
data_path = 'IEMOCAP_full_release/Session*/sentences/wav/Ses*/*.wav'
# files = glob.glob(data_path)
# file_name = [x.split(os.sep)[-1].split('.')[0] for x in files]

In [5]:
csv_file_path_reg = 'IEMOCAP_full_release/Session*/dialog/EmoEvaluation/Categorical/Ses*.txt'

In [6]:
dialogues_path_reg = 'IEMOCAP_full_release/Session*/dialog/transcriptions/Ses*.txt'
dialogues_paths = glob.glob(dialogues_path_reg)

In [7]:
csv_files = glob.glob(csv_file_path_reg)

In [8]:
cat_df = get_meta(csv_file_path_reg, 'category')
dial_df  =get_meta(dialogues_path_reg, 'dial')

NameError: name 'get_meta' is not defined

In [ ]:
cat_df['category'] = cat_df['category'].str.replace(';.*','')

In [14]:
cat_df[cat_df['file']=='Ses04F_script02_2_F000']

,file,category
134,Ses04F_script02_2_F000,anger
3892,Ses04F_script02_2_F000,frustration
4985,Ses04F_script02_2_F000,other


In [25]:
meta_df = pd.merge(cat_df, dial_df, on = 'file')

In [42]:
meta_df = meta_df.groupby(['file','dial']).head(1).reset_index().drop(columns=['index'])

In [44]:

class LighteningModel(pl.LightningModule):
    def __init__(self,  num_classes=10):
        super(LighteningModel, self).__init__()
        self.train_r2 = torchmetrics.R2Score()
        self.val_r2 = torchmetrics.R2Score()
        self.train_auc = torchmetrics.classification.AUROC(task = 'multiclass', num_classes=num_classes)
        self.val_auc = torchmetrics.classification.AUROC(task = 'multiclass', num_classes=num_classes)
        
        self.val_f1 = torchmetrics.classification.F1Score(task = 'multiclass', num_classes=num_classes)
        self.train_f1 = torchmetrics.classification.F1Score(task = 'multiclass', num_classes=num_classes)
        
        
        feature_extractor = AutoFeatureExtractor.from_pretrained("ast-finetuned-audioset-10-10-0.4593")
        self.sp_model = ASTForAudioClassification.from_pretrained("ast-finetuned-audioset-10-10-0.4593", return_dict=False)
        
        self.bert_model = BertModel.from_pretrained("bert-base-uncased")
        
        for param in self.sp_model.parameters():
            param.requires_grad = False
        self.sp_model.classifier.dense = nn.Linear(768,256)
        
        for param in self.bert_model.parameters():
            param.requires_grad = False
            
        for param in self.bert_model.encoder.layer[-1:].parameters():
            param.requires_grad = True
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
        self.fc1 = nn.Linear(1024, num_classes)
        self.dropout = nn.Dropout(.7) 
        self.relu = nn.ReLU()
    def forward(self,x,y):
        x = x['input_values'].view(x['input_values'].size(0), 1024,128)
        input_bert = y['input_ids'].view(y['input_ids'].size(0), 50)
        atten_bert =  y['attention_mask'].view(y['attention_mask'].size(0), 50)
#         y = y['input_ids']
#         print(y.shape)
        sp = self.sp_model(x, return_dict=False)[0] #256
        bert = self.bert_model(input_ids = input_bert, attention_mask = atten_bert, return_dict = False)[0]
        bert = self.dropout(bert.mean(1))
        sp = self.dropout(sp)

        out = torch.concat([bert, sp], axis = 1)
        out = self.dropout(out)
        out = self.fc1(out)
        
        return out
    
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=0.0001, weight_decay=1e-3)
        return optimizer
    
    
    def training_step(self, train_batch, batch_idx):
        signal, dial, labels = train_batch
#         signal = signal['input_values'].view(signal['input_values'].size(0), 1024,128).shape
        labels =labels.float().to('cuda:0')
        outputs = self(signal, dial).to('cuda:0')#.argmax(1).float()
        criterion = torch.nn.CrossEntropyLoss()
        loss = criterion(outputs, labels.long())
        self.train_auc(outputs, labels.int())
        self.train_f1(outputs, labels.int())
        
        self.log('loss', loss, on_step=False, on_epoch=True)
        self.log('train_auc', self.train_auc, on_step=False, on_epoch=True)
        self.log('train_f1', self.train_f1, on_step=False, on_epoch=True)
        
        
        return loss
    
    def validation_step(self, val_batch, batch_idx):
        signal ,dial, labels = val_batch
        labels = labels.float().to('cuda:0')
#         signal = signal['input_values'].view(signal['input_values'].size(0), 1024,128).shape
        
        outputs = self(signal, dial).to('cuda:0')#.argmax(1).float()
        criterion = torch.nn.CrossEntropyLoss()
        
        loss = criterion(outputs, labels.long())
        self.val_auc(outputs, labels.int())
        self.val_f1(outputs, labels.int())
        
        self.log('val_loss', loss, on_step=False, on_epoch=True)
        self.log('val_auc', self.val_auc, on_step=False, on_epoch=True)
        self.log('val_f1', self.val_f1, on_step=False, on_epoch=True)



In [45]:
model = LighteningModel()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [46]:
next(model.parameters()).is_cuda

False

In [47]:
BATCH_SIZE = 16

In [48]:
dataset = AudioDataset(meta_df,data_path, modality = 'all')

In [49]:
len(dataset)

10039

In [50]:
meta_df.shape

(10039, 3)

In [51]:
train_set, val_set = torch.utils.data.random_split(dataset, [0.8, 0.2],)

In [52]:
# sampler = WeightedRandomSampler(sample_weights, int(len(train_set)*1.5), replacement=True)
train_dataloader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle = True, num_workers=8, drop_last=True, )
test_dataloader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=8, drop_last=True)

In [53]:
trainer = pl.Trainer( max_epochs=500, gradient_clip_val=0, logger = llogger, )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, train_dataloader, test_dataloader, )

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name       | Type                      | Params
----------------------------------------------------------
0  | train_r2   | R2Score                   | 0     
1  | val_r2     | R2Score                   | 0     
2  | train_auc  | MulticlassAUROC           | 0     
3  | val_auc    | MulticlassAUROC           | 0     
4  | val_f1     | MulticlassF1Score         | 0     
5  | train_f1   | MulticlassF1Score         | 0     
6  | sp_model   | ASTForAudioClassification | 86.4 M
7  | bert_model | BertModel                 | 109 M 
8  | fc1        | Linear                    | 10.2 K
9  | dropout    | Dropout                   | 0     
10 | relu       | ReLU                      | 0     
----------------------------------------------------------
7.3 M     Trainable params
188 M     Non-trainable params
195 M     Total params
783.513   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [28]:
meta_df

,file,category,dial
0,Ses01F_impro01_F000,neutral state,Excuse me.
1,Ses01F_impro01_F001,neutral state,Yeah.
2,Ses01F_impro01_F002,neutral state,Is there a problem?
3,Ses01F_impro01_F002,surprise,Is there a problem?
4,Ses01F_impro01_F003,anger,You did.
...,...,...,...
9984,Ses05M_script03_2_M041,anger,"You are a vile tempered, wicked living, evil ..."
9985,Ses05M_script03_2_M042,anger,"Oh, you're not going like this."
9986,Ses05M_script03_2_M043,anger,"[GARBAGE] No, you're not."
9987,Ses05M_script03_2_M044,anger,oh! Marry you again? I wouldn't marry you aga...
